## Initialize tokens and Slice Manager

Display Existing Resource Availability

The user can produce a map depicting the all of the currently available resources

The map that is drawn should have a style that is consistent with the other maps notebooks.

Desirable attributes of the map:
- Pin drop for each site
- Links between sites where they exist
- Click/hover on site to get more detailed info
-Ability to click/hover on links to get more info.

If possible, the user should have the ability to save the map image to a file that can be imported later into presentations or papers.

In [13]:
import json
import os
from fabrictestbed.slice_manager import SliceManager, Status

credentialmanager_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']

print(f"CM Host: {credentialmanager_host} Orchestrator Host: {orchestrator_host}")
print()

slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credentialmanager_host, project_name='all', scope='all')
slice_manager.initialize()

ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()
    
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()
print()

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTCNBTGYYGKY3GMI4GIOBSGFSDEODEHE2TMMBYMVSDMNZVG5TGMZB7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRYG44TMOBVGE4DAMBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)




## Get information via slice_manager.resources()

In [14]:
from fabrictestbed.slice_editor import Capacities

status, advertised_topology = slice_manager.resources()

#get information from the advertised topology to display on the graph

    #used info from the following documentation sites for retrieving attributes:
        #https://github.com/fabric-testbed/InformationModel/blob/911aed03256ed2c4a9f2dbbddb693f1dc47ed8ab/fim/graph/slices/networkx_asm.py#L33
        #https://github.com/fabric-testbed/InformationModel/blob/master/fim/user/topology.py

#containters        
topology_dict = {}
site_dict = {}

for t in advertised_topology.sites.values():
    
    #name
    name = t.name 
    
    #location for site_dict
    loc_info = t.get_property("location")
    if loc_info is not None:
        loc = loc_info.to_latlon()
        site_dict[name] = loc
    
    #capacities for html
    #available capacities total vs. currently allocated ones
    available_capacities = t.get_property("capacities")
    allocated_capacities = t.get_property("capacity_allocations")
    
    if allocated_capacities is None:
        allocated_capacities = Capacities() 
        
    if available_capacities is None:
        available_capacities = Capacities()
        
    d1 = available_capacities.__dict__.copy()
    d2 = allocated_capacities.__dict__.copy()
    
    #remove capacities that are not available 
    for d in allocated_capacities.__dict__:
        if d1[d] == 0 and d2[d] == 0:
            d1.pop(d)
            d2.pop(d)
    
    #html string with info for graph
    html_str = '<center><b>' + name + '</b></center>'
    html_str += '<b>capacities:</b><br>'

    for c in d1:
        html_str = html_str + c + ': ' + str(d1[c] - d2[c]) + '/' + str(d1[c]) + ' ' + Capacities.UNITS[c]
        if c == 'unit':
            continue
        html_str = html_str + ', '
        
    html_str += '<br><b>components: </b><br>'
    
    for c in t.components.values():
        html_str += '&emsp; <i>' + c.name + '</i> ('
        available = c.get_property("capacities")
        allocated = c.get_property("capacity_allocations")
        
        if allocated is None:
            allocated = Capacities()
        
        if available is None:
            available = Capacities()
                
        d1 = available.__dict__.copy()
        d2 = allocated.__dict__.copy()
    
        #remove capacities that are not available 
        for d in allocated.__dict__:
            if d1[d] == 0 and d2[d] == 0:
                d1.pop(d)
                d2.pop(d)
        for c in d1:
            html_str = html_str  + c + ': ' + str(d1[c] - d2[c]) + '/' + str(d1[c]) + ' ' + Capacities.UNITS[c] + ' '
    
        html_str += ')<br>' 
    
    topology_dict[t.name] = html_str
    print(html_str)

<center><b>UKY</b></center><b>capacities:</b><br>cpu: 6/6 , core: 178/192 , ram: 1490/1536 G, disk: 9430/9600 G, unit: 3/3 <br><b>components: </b><br>&emsp; <i>GPU-RTX6000</i> (unit: 2/2  )<br>&emsp; <i>GPU-Tesla T4</i> (unit: 4/4  )<br>&emsp; <i>NVME-P4510</i> (disk: 10000/10000 G unit: 10/10  )<br>&emsp; <i>SharedNIC-ConnectX-6</i> (unit: 381/381  )<br>&emsp; <i>SmartNIC-ConnectX-6</i> (unit: 2/2  )<br>&emsp; <i>SmartNIC-ConnectX-5</i> (unit: 2/2  )<br>
<center><b>RENC</b></center><b>capacities:</b><br><br><b>components: </b><br>
<center><b>STAR</b></center><b>capacities:</b><br><br><b>components: </b><br>
<center><b>LBNL</b></center><b>capacities:</b><br>cpu: 6/6 , core: 173/192 , ram: 1452/1536 G, disk: 9240/9600 G, unit: 3/3 <br><b>components: </b><br>&emsp; <i>SharedNIC-ConnectX-6</i> (unit: 379/381  )<br>&emsp; <i>NVME-P4510</i> (disk: 9000/10000 G unit: 9/10  )<br>&emsp; <i>GPU-RTX6000</i> (unit: 2/2  )<br>&emsp; <i>GPU-Tesla T4</i> (unit: 4/4  )<br>&emsp; <i>SmartNIC-ConnectX-

## Map

In [15]:
from ipywidgets import HTML, Button, Layout, HBox, VBox, embed
from ipyleaflet import Map, basemaps, Marker, Popup, ImageOverlay, SearchControl, DivIcon, MarkerCluster, Polyline, AntPath, AwesomeIcon

sites = Map(basemap=basemaps.Esri.WorldStreetMap, center=(38.12480976137421, -95.7129), zoom = 3.8)

server_icon = AwesomeIcon(
    name='server',
    marker_color='blue',
    icon_color='white',
    spin=False
)

#display sites
for site, coordinates in site_dict.items():
    if site in topology_dict.keys():
        b = HTML(value = topology_dict[site], disabled=True)
        html_info = HBox([b], layout=Layout(height='160px', overflow_y='auto'))
    else:
        message = '<b>' + site + '</b>' #cause at the moment only one site is actually up
        html_info = HTML(message)
        image = ImageOverlay(url = "https://jupyter-beta.fabric-testbed.net/user/nannkat@bu.edu/files/jupyter-examples/fabric/nanna/red_x.png?_xsrf=2%7Cd410e56b%7C9c22d80ad6d1605024c3f11ce9b3d548%7C1622770101", 
                             bounds = ((coordinates[0]-1,coordinates[1]-1.5),(coordinates[0]+1,coordinates[1]+1.5)))
        sites.add_layer(image)
    marker = Marker(location = coordinates, draggable = False, title = site, rise_on_hover = True, icon = server_icon)
    marker.popup = html_info
    sites.add_layer(marker)

#display links
for link in advertised_topology.links.values():
  
    inter1 = link._interfaces[0]
    inter2 = link._interfaces[1]
    places = inter1.name
    
    site1, site2 = places.split("_")
    html_string = '<center>LINK</center>'
    html_string += '<b>layer:</b> ' + link.get_property('layer') + '<br>'
    html_string += '<b>type:</b> ' + str(link.get_property('type')) + '<br>'
    html_string += '<b>sites:</b> ' + site1 + ' and ' + site2 + '<br>'
    html_string += '<b>capacities:</b> bw: ' + str(inter1.get_property('capacities').bw) + ' Gbps (' + inter1.name + ') / ' + str(inter2.get_property('capacities').bw) + ' Gbps (' + inter2.name + ')<br>'
    html_string += '<b>name:</b> ' + link.name + '<br>'
    
    ant_path = AntPath(
       dash_array=[1, 10],
        delay=1000,
        color='#FFD700',
        pulse_color = '#FFD700')
    
    ant_path.locations = [[site_dict[site1],site_dict[site2]]]
    
    link_inf = HTML(value = html_string, disabled=True)
   
    html_content = VBox([link_inf], layout = Layout(width = '300px', overflow_x = 'auto'))
    ant_path.popup = html_content
    sites.add_layer(ant_path)

#IF you prefer a normal line instead of ant-path style:
    #To do a normal line popup:
        #line = Polyline(
        # color = "green" ,
        # fill= False,
        # weight = 3)
        #line.locations = [[site_dict[site1],site_dict[site2]]]
        #line.popup = html_content
        #sites.add_layer(line)

sites

Map(center=[38.12480976137421, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…

## Save as HTML file

In [7]:
from ipywidgets import embed

embed.embed_minimal_html('map.html', views=[sites])